In [1]:
import pandas as pd
import numpy as np
from recordlinkage.preprocessing import clean
from recordlinkage.preprocessing import phonetic
from recordlinkage.index import Full
from recordlinkage.index import Block
import recordlinkage as rl
from metaphone import doublemetaphone

In [2]:
#Opening the records
main_org = pd.read_excel('Main Records.xlsx')
main = main_org.copy()
dup_org = pd.read_excel('Duplicates.xlsx')
dup=dup_org.copy()

In [3]:
main.head()

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue)
0,1f27cdd1,Schwartz,Daniel,12/09/2013,NaN,NaN,Pentecostalism and other Charismatic,NaN,White,NaN,NaN,English
1,3bb0e015,Wishart,Ray,07/24/2007,NaN,NaN,Sikhism,NaN,Southeast Asian,NaN,NaN,NaN
2,4b532e78,Woods,Shirley,07/10/2020,NaN,NaN,United Church,NaN,East Asian,NaN,NaN,Korean
3,612a0deb,Germon,Tom,09/06/2014,NaN,Indian,NaN,Indian,Indian,Canada,Canada,NaN
4,5c1ac840,Williams,Betty,04/28/2020,NaN,NaN,NaN,NaN,Black,Canada,Canada,English


In [14]:
len(main)

20001

In [4]:
#Function to fill in null values
def clean_file(df):
    #Ethnicity == Race found
    #Race = Indigenous Identity
    #If not a citizen of Canada, citizenship -> birth place
    # If race = 'Alaska Native', 'American Indian', 'Native Hawaiian','Pacific Islander' -> Citizen = Canada
    race_list = ['Alaska Native', 'American Indian', 'Native Hawaiian','Pacific Islander']
    r = df['Race']
    ig = df['Indigenous Identity']
    e = df['Ethnicity']
    c = df['Citizenship']
    p = df['Place of Birth']
    r_i = df['Race'].isna()
    e_i = df['Ethnicity'].isna()
    i_i = df['Indigenous Identity'].isna()
    c_i = df['Citizenship'].isna()
    p_i = df['Place of Birth'].isna()

    for i in range(df.shape[0]):
        if ((r_i[i] == True) or (i_i[i] == True) or (e_i[i] == True)):
            if ((r_i[i] == False) or (i_i[i] == False) or (e_i[i] == False)):
                if r_i[i] == False:
                    if e_i[i] == True:
                        e[i] = r[i]
                    if i_i[i] == True:
                        ig[i] = r[i]
                if e_i[i] == False:
                    if r_i[i] == True:
                        r[i] = e[i]
                    if i_i[i] == True:
                        ig[i] = e[i]
                if i_i[i] == False:
                    if r_i[i] == True:
                        r[i] = ig[i]
                    if e_i[i] == True:
                        e[i] = ig[i]

        if r_i[i] == False:
            if r_i[i] in race_list:
                if c_i[i] == True:
                    c[i] = 'Canada'

        if c_i[i] == False:
            if c_i[i] != 'Canada':
                if p_i[i] == True:
                    p[i] = c[i] 
                
    return r,ig,e,c,p

In [5]:
a,b,c,d,e = clean_file(main)
main['Race'] = a
main['Indigenous Identity'] = b
main['Ethnicity'] = c
main['Citizenship'] = d
main['Place of Birth'] = e 

In [6]:
a,b,c,d,e = clean_file(dup)
dup['Race'] = a
dup['Indigenous Identity'] = b
dup['Ethnicity'] = c
dup['Citizenship'] = d
dup['Place of Birth'] = e 

C:\Users\hamza\AppData\Local\Temp\ipykernel_14388\1021505405.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[i] = e[i]


In [7]:
#Inserting the following columns
cols = main.columns.values
cols=np.insert(cols, 0,'Phon Mother Name')
cols=np.insert(cols, 0,'Phon First Name')
cols=np.insert(cols, 0,'Phon Last Name')

In [8]:
method = 'metaphone'
clean_main_last = clean (main["Last Name"])
clean_main_first = clean(main['First Name'])
clean_main_mother = clean(main["Mother's Birth Last Name"] )
clean_main_last = phonetic(clean_main_last, method = method, concat=True, encoding='utf-8', decode_error='strict')
clean_main_first = phonetic(clean_main_first, method = method, concat=True, encoding='utf-8', decode_error='strict')
clean_main_mother = phonetic(clean_main_mother, method = method, concat=True, encoding='utf-8', decode_error='strict')
clean_main = pd.concat([clean_main_last, clean_main_first, clean_main_mother, main], axis = 1)
clean_main.columns = cols

In [9]:
method = 'metaphone'
clean_dup_last = clean (dup["Last Name"])
clean_dup_first = clean (dup["First Name"])
clean_dup_mother = clean (dup["Mother's Birth Last Name"])
clean_dup_last = phonetic(clean_dup_last, method = method, concat=True, encoding='utf-8', decode_error='strict')
clean_dup_first = phonetic(clean_dup_first, method = method, concat=True, encoding='utf-8', decode_error='strict')
clean_dup_mother = phonetic(clean_dup_mother, method = method, concat=True, encoding='utf-8', decode_error='strict')
clean_dup = pd.concat([clean_dup_last, clean_dup_first, clean_dup_mother, dup], axis = 1)
clean_dup.columns = cols

In [11]:
#Block Index
indexer = rl.Index()
indexer.add(Block('First Name', 'First Name'))
indexer.add(Block('Last Name', 'Last Name'))
indexer.add(Block("Mother's Birth Last Name", "Mother's Birth Last Name"))
indexer.add(Block('Last Name', "Mother's Birth Last Name"))
indexer.add(Block( "Mother's Birth Last Name", 'Last Name'))
indexer.add(Block('Phon First Name', 'Phon First Name'))
indexer.add(Block('Phon Last Name', 'Phon Last Name'))
indexer.add(Block("Phon Mother Name", "Phon Mother Name"))
indexer.add(Block('Phon Last Name', "Phon Mother Name"))
indexer.add(Block( "Phon Mother Name", 'Phon Last Name'))
candidate_links = indexer.index(clean_main, clean_dup)


In [12]:
len(candidate_links)

1129103

In [12]:
#Comparing the records
name_th = 0.85
method = "levenshtein"
missing = 0
disagree = -1000
compare_cl = rl.Compare()
compare_cl.exact('Last Name', 'Last Name',label="Last Name")
compare_cl.exact('First Name', 'First Name', label="First Name")
compare_cl.exact("Mother's Birth Last Name", "Mother's Birth Last Name", label="Mother's Birth Last Name")
compare_cl.exact("Mother's Birth Last Name", 'Last Name', label="Main-Mother->Last")
compare_cl.exact('Last Name', "Mother's Birth Last Name",  label="Main-Last->Mother")
compare_cl.exact('Phon Last Name', 'Phon Last Name',label="Phon Last Name", disagree_value = disagree)
compare_cl.exact('Phon First Name', 'Phon First Name', label="Phon First Name", disagree_value = disagree)
compare_cl.exact("Phon Mother Name", "Phon Mother Name", label="Phon Mother's Birth Last Name")
compare_cl.exact("Phon Mother Name", 'Phon Last Name', label="Phon Main-Mother->Last")
compare_cl.exact('Phon Last Name', "Phon Mother Name",  label="Phon Main-Last->Mother")
compare_cl.exact("Race", "Race", label="Race")
compare_cl.exact("Religion", "Religion", label="Religion")
compare_cl.exact("Citizenship", "Citizenship", label="Citizenship")
compare_cl.exact("Language (Mother Tongue)", "Language (Mother Tongue)", label="Language")
compare_cl.exact("Date of Birth", "Date of Birth", label="DOB")
compare_cl.string("Date of Birth", "Date of Birth",method=method, threshold=name_th, label="fuzzy DOB")
features = compare_cl.compute(candidate_links, clean_main,clean_dup)

In [13]:
#Accounting for Last Name not matching with Last Name, but matching with Mother's Last Name
pln = features['Phon Last Name'].reset_index(drop=True)
pmn = features["Phon Mother's Birth Last Name"].reset_index(drop=True)
pmnl = features["Phon Main-Mother->Last"].reset_index(drop=True)
pmnr = features["Phon Main-Last->Mother"].reset_index(drop=True)
adj = []

for i in range(len(pln)):
    if pln[i]<0:
        if pmn[i] + pmnl[i] + pmnr[i] >  0:
            adj.append(0)
        else:
            adj.append(-1000)
    else:
        adj.append(pln[i])


In [14]:
features['Phon Last Name'] = adj

In [15]:
#Looking at features where there is atleast one matching feature, and no features that disagree
found = features.copy()
found['sum'] = features.sum(axis = 1)
found = found[found['sum']> 1]
col_n = found.columns.values
col_n=np.insert(col_n, 0,'Dup Index')
col_n=np.insert(col_n, 0,'Main Index')
found = found.reset_index()
found.columns = col_n

In [16]:
found

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum
0,0,82,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0
1,0,440,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,3.0
2,0,1013,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0
3,0,1165,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,2.0
4,0,1213,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157834,20000,1472,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0
157835,20000,1491,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0
157836,20000,3817,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,2.0
157837,20000,8252,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,2.0


In [17]:
#Creating the Weight Matrix
weight_sum = []
l = found['First Name']
f = found['Last Name']
mn = found["Mother's Birth Last Name"]
mnl = found["Main-Mother->Last"]
mnr = found["Main-Last->Mother"]
pl = found['Phon First Name']
pf = found['Phon Last Name']
pmn = found["Phon Mother's Birth Last Name"]
pmnl = found["Phon Main-Mother->Last"]
pmnr = found["Phon Main-Last->Mother"]
e = found['Race']
r = found['Religion']
t = found['Language']
c = found['Citizenship']
d = found['fuzzy DOB']
d1 = found['DOB']

for i in range(0, found.shape[0]):
    x = 0
    y = 0
    z = d1[i] * 10
    if c[i] + r[i] + t[i] + e[i] == 4:
        x = 10
    elif c[i] + r[i] + t[i] + e[i] == 3:
        x = 5
    elif c[i] + r[i] + t[i] + e[i] == 2:
        x = 2
    elif c[i] + r[i] + t[i] + e[i] == 1:
        x = 1

        
        
    if f[i] + l[i] + d[i] + mn[i] == 4:
        y = 80
    elif f[i] + pl[i] + d[i] + mn[i] == 4:
        y = 75
    elif pf[i] + l[i] + d[i] + mn[i] == 4:
        y = 75
    elif f[i] + l[i] + d[i] + pmn[i] == 4:
        y = 75
    elif f[i] + l[i] + d[i] + pmnl[i] == 4:
        y = 70
    elif f[i] + l[i] + d[i] + pmnr[i] == 4:
        y = 70
    elif pf[i] + pl[i] + d[i] + mn[i] == 4:
        y = 65
    elif f[i] + l[i] + d[i] == 3:
        y = 65
    elif pf[i] + l[i] + d[i] + pmn[i] == 4:
        y = 60
    elif f[i] + pl[i] + d[i] + pmn[i] == 4:
        y = 60
    elif f[i] + mn[i] + d[i] == 3:
        y = 60
    elif f[i] + pl[i] + d[i] == 3:
        y = 60
    elif pf[i] + l[i] + d[i] == 3:
        y = 60
    elif pf[i] + l[i] + d[i] + pmnl[i] == 4:
        y = 55
    elif pf[i] + l[i] + d[i] + pmnr[i] == 4:
        y = 55
    elif f[i] + mnl[i] + d[i] == 3:
        y = 55
    elif f[i] + mnr[i] + d[i] == 3:
        y = 55
    elif pf[i] + mn[i] + d[i] == 3:
        y = 55
    elif f[i] + l[i] == 2:
        y = 55
    elif f[i] + pl[i] + d[i] + pmnl[i] == 4:
        y = 50
    elif f[i] + pl[i] + d[i] + pmnr[i] == 4:
        y = 50
    elif l[i] + mn[i] + d[i] == 3:
        y = 50
    elif f[i] + pmn[i] + d[i] == 3:
        y = 50
    elif pl[i] + mn[i] + d[i] == 3:
        y = 50
    elif pf[i] + mnl[i] + d[i] == 3:
        y = 50
    elif pf[i] + mnr[i] + d[i] == 3:
        y = 50
    elif pl[i] + pmn[i] + d[i] == 3:
        y = 45
    elif pl[i] + mn[i] + d[i] == 3:
        y = 45
    elif f[i] + mn[i]== 2:
        y = 45
    elif f[i] + pl[i] == 2:
        y = 45
    elif pf[i] + l[i] == 2:
        y = 45
    elif pf[i] + pl[i] == 2:
        y = 40
    elif f[i] + mnl[i]== 2:
        y = 35
    elif f[i] + mnr[i]== 2:
        y = 35
    elif l[i] + mn[i]== 2:
        y = 35
    elif f[i] + pmn[i]== 2:
        y = 35
    elif f[i] + pmnl[i]== 2:
        y = 30
    elif pf[i] + mn[i]== 2:
        y = 30
    elif f[i] + d[i]== 2:
        y = 25
    elif l[i] + d[i]== 2:
        y = 25
    elif f[i] + pmnr[i]== 2:
        y = 25
    elif f[i] + pmnl[i]== 2:
        y = 25
    elif pf[i] + pmnl[i]== 2:
        y = 25
    elif pl[i] + mn[i]== 2:
        y = 25
    elif pf[i] + pmnr[i]== 2:
        y = 20
    elif pf[i] + pmnl[i]== 2:
        y = 20
    elif l[i] + pmn[i]== 2:
        y = 20
    elif d[i] + mn[i]== 2:
        y = 15
    elif pf[i] + d[i]== 2:
        y = 15
    elif pl[i] + d[i]== 2:
        y = 15
    elif d[i] + pmn[i]== 2:
        y = 10
    elif d[i] + pmnr[i]== 2:
        y = 5
    elif d[i] + pmnl[i]== 2:
        y = 5
    
    w=x+y+z
    
    weight_sum.append(w)

In [18]:
#Adding Weight Feature
found['weight'] = weight_sum
found

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,0,82,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0,0
1,0,440,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,3.0,1
2,0,1013,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0,0
3,0,1165,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,2.0,1
4,0,1213,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157834,20000,1472,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0,0
157835,20000,1491,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0,0
157836,20000,3817,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,2.0,0
157837,20000,8252,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,2.0,0


In [19]:
match = found[found['weight']> 5]
print(len(match))

15129


In [20]:
match

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
26,6,226,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
94,21,883,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
187,36,8300,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,2.0,15
244,46,4290,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,4.0,35
245,46,8518,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,4.0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157814,19996,10136,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,4.0,35
157815,19997,10137,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
157825,19998,7301,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0.0,3.0,40
157826,19998,10138,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,4.0,35


In [21]:
#match.sort_values(by = ['sum'])
match.sort_values(by = ['weight'], ascending = False)

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
46372,6032,539,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1.0,15.0,95
85238,11156,1913,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1.0,15.0,95
138813,17712,7852,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1.0,14.0,92
101674,13252,3392,1,1,1,0,0,1,1,1,1,1,0,0,1,1,1,1.0,12.0,92
77450,10129,1629,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1.0,14.0,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32141,4301,474,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
31970,4277,1917,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
147441,18765,1638,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
31793,4255,875,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15


In [22]:
joins = match.sort_values(by = ['Dup Index','weight'], ascending = [True,False])
joins

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
29878,4004,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1.0,5.0,36
29897,4008,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1.0,5.0,36
29919,4014,2,1,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1.0,6.0,71
29922,4017,3,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
29965,4021,4,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,1.0,6.0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110116,14269,10137,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
157815,19997,10137,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
157826,19998,10138,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,4.0,35
134306,17161,10138,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25


In [23]:
upper = joins[joins['weight']> 55]
lower = joins[joins['weight']< 60]
upper =upper.reset_index(drop = True)
lower =lower.reset_index(drop = True)

In [24]:
#Finding Duplicate Records with High Quality where only one match was found with the Duplicate Record
#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
unq, unq_idx, unq_cnt = np.unique(upper['Dup Index'], return_inverse=True, return_counts=True)
cnt_mask = unq_cnt > 1
dup_ids = unq[cnt_mask]

#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
cnt_idx, = np.nonzero(cnt_mask)
idx_mask = np.in1d(unq_idx, cnt_idx)
idx_idx, = np.nonzero(idx_mask)
srt_idx = np.argsort(unq_idx[idx_mask])
dup_idx = np.split(idx_idx[srt_idx], np.cumsum(unq_cnt[cnt_mask])[:-1])

In [25]:
#Higher weight always better found for upper
'''
test = upper.drop(['Main Index','Dup Index', 'sum', 'weight'], axis = 1)
for i in range(len(dup_idx)):
    ind = dup_idx[i]
    x = ind[0]
    y = ind[1]
    if ind[0]>ind[1]:
        y = ind[0]
        x = ind[1]

    
    for j in range(test.shape[1]):
        if test.loc[x][j] == 0:
            if test.loc[y][j] == 1:
                    print ('Column:', test.columns[j],'big index: ', x, 'small index: ', y)
'''

"\ntest = upper.drop(['Main Index','Dup Index', 'sum', 'weight'], axis = 1)\nfor i in range(len(dup_idx)):\n    ind = dup_idx[i]\n    x = ind[0]\n    y = ind[1]\n    if ind[0]>ind[1]:\n        y = ind[0]\n        x = ind[1]\n\n    \n    for j in range(test.shape[1]):\n        if test.loc[x][j] == 0:\n            if test.loc[y][j] == 1:\n                    print ('Column:', test.columns[j],'big index: ', x, 'small index: ', y)\n"

In [26]:
id_count = [0] * dup.shape[0]
main_id = [[-1 for i in range(1)] for i in range(dup.shape[0])] 
quality = [[0 for i in range(1)] for i in range(dup.shape[0])] 
unique = [0] * dup.shape[0]
iteration = [-1] * dup.shape[0]

In [27]:
#Removing matched duplicate records
to_drop = []
for i in range(len(dup_idx)):
    a = dup_idx[i]
    k = np.argmin(a)
    for j in range(len(a)):
        if j!=k:
            to_drop.append(a[j])

upper = upper.drop(index = to_drop)

In [28]:
#Adding the matched records
filler = upper.reset_index(drop = True)
for index in range(filler.shape[0]):
    i = filler['Dup Index'].loc[index]
    main_id[i][0] = filler['Main Index'].loc[index]
    id_count[i] = 1
    unique[i] = 1
    iteration[i] = 1
    quality[i][0] = filler['weight'].loc[index]

In [29]:
upper['Dup Index'].values

array([    2,     7,    15, ...,  9975, 10003, 10035], dtype=int64)

In [30]:
dup_done = upper['Dup Index'].values
to_drop = []
for i in range(lower.shape[0]):
    if lower['Dup Index'].loc[i] in dup_done:
        to_drop.append(i)

In [31]:
c = 0
for i in range(len(id_count)):
    if id_count[i] == 1:
        c = c+ 1

print (c)

5216


In [32]:
lower = lower.drop(index = to_drop)
lower =lower.reset_index(drop = True)
lower

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,4004,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1.0,5.0,36
1,4008,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1.0,5.0,36
2,4017,3,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
3,4021,4,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,1.0,6.0,37
4,4022,5,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8956,14269,10137,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
8957,19997,10137,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
8958,19998,10138,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,4.0,35
8959,17161,10138,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25


In [33]:
#Finding unique matches between the main records and duplicate records
#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
unq, unq_idx, unq_cnt = np.unique(lower['Dup Index'], return_inverse=True, return_counts=True)
cnt_mask = unq_cnt == 1
unq_ids = unq[cnt_mask]

#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
cnt_idx, = np.nonzero(cnt_mask)
idx_mask = np.in1d(unq_idx, cnt_idx)
idx_idx, = np.nonzero(idx_mask)
srt_idx = np.argsort(unq_idx[idx_mask])
unq_idx = np.split(idx_idx[srt_idx], np.cumsum(unq_cnt[cnt_mask])[:-1])

In [34]:
#Removing matched duplicate records
to_drop = []
for i in range(len(unq_idx)):
    to_drop.append(unq_idx[i][0])

In [35]:
#Adding found matches
filler = lower
for index in to_drop:
    i = filler['Dup Index'].loc[index]
    main_id[i][0] = filler['Main Index'].loc[index]
    iteration[i] = 2
    unique[i] = 1
    id_count[i] = 1
    quality[i][0] = filler['weight'].loc[index]

In [36]:
c = 0
for i in range(len(id_count)):
    if id_count[i] == 1:
        c = c+ 1
print (c)

7946


In [37]:
found_2 = features.copy()
found_2['sum'] = features.sum(axis = 1)
col_n = found_2.columns.values
col_n=np.insert(col_n, 0,'Dup Index')
col_n=np.insert(col_n, 0,'Main Index')
found_2 = found_2.reset_index()
found_2.columns = col_n

In [38]:
#Creating a modeified weight matrix
weight_sum_2 = []
l = found_2['First Name']
f = found_2['Last Name']
mn = found_2["Mother's Birth Last Name"]
mnl = found_2["Main-Mother->Last"]
mnr = found_2["Main-Last->Mother"]
pl = found_2['Phon First Name']
pf = found_2['Phon Last Name']
pmn = found_2["Phon Mother's Birth Last Name"]
pmnl = found_2["Phon Main-Mother->Last"]
pmnr = found_2["Phon Main-Last->Mother"]
e = found_2['Race']
r = found_2['Religion']
t = found_2['Language']
c = found_2['Citizenship']
d = found_2['fuzzy DOB']
d1 = found_2['DOB']

for i in range(0, found_2.shape[0]):
    x = 0
    y = 0
    z = d1[i] * 10
    if c[i] + r[i] + t[i] + e[i] == 4:
        x = 10
    elif c[i] + r[i] + t[i] + e[i] == 3:
        x = 5
    elif c[i] + r[i] + t[i] + e[i] == 2:
        x = 2
    elif c[i] + r[i] + t[i] + e[i] == 1:
        x = 1

        
        
    if f[i] + l[i] + d[i] + mn[i] == 4:
        y = 80
    elif f[i] + pl[i] + d[i] + mn[i] == 4:
        y = 75
    elif pf[i] + l[i] + d[i] + mn[i] == 4:
        y = 75
    elif f[i] + l[i] + d[i] + pmn[i] == 4:
        y = 75
    elif f[i] + l[i] + d[i] + pmnl[i] == 4:
        y = 70
    elif f[i] + l[i] + d[i] + pmnr[i] == 4:
        y = 70
    elif pf[i] + pl[i] + d[i] + mn[i] == 4:
        y = 65
    elif f[i] + l[i] + d[i] == 3:
        y = 65
    elif pf[i] + l[i] + d[i] + pmn[i] == 4:
        y = 60
    elif f[i] + pl[i] + d[i] + pmn[i] == 4:
        y = 60
    elif f[i] + mn[i] + d[i] == 3:
        y = 60
    elif f[i] + pl[i] + d[i] == 3:
        y = 60
    elif pf[i] + l[i] + d[i] == 3:
        y = 60
    elif pf[i] + l[i] + d[i] + pmnl[i] == 4:
        y = 55
    elif pf[i] + l[i] + d[i] + pmnr[i] == 4:
        y = 55
    elif f[i] + mnl[i] + d[i] == 3:
        y = 55
    elif f[i] + mnr[i] + d[i] == 3:
        y = 55
    elif pf[i] + mn[i] + d[i] == 3:
        y = 55
    elif f[i] + l[i] == 2:
        y = 55
    elif f[i] + pl[i] + d[i] + pmnl[i] == 4:
        y = 50
    elif f[i] + pl[i] + d[i] + pmnr[i] == 4:
        y = 50
    elif l[i] + mn[i] + d[i] == 3:
        y = 50
    elif f[i] + pmn[i] + d[i] == 3:
        y = 50
    elif pl[i] + mn[i] + d[i] == 3:
        y = 50
    elif pf[i] + mnl[i] + d[i] == 3:
        y = 50
    elif pf[i] + mnr[i] + d[i] == 3:
        y = 50
    elif pl[i] + pmn[i] + d[i] == 3:
        y = 45
    elif pl[i] + mn[i] + d[i] == 3:
        y = 45
    elif f[i] + mn[i]== 2:
        y = 45
    elif f[i] + pl[i] == 2:
        y = 45
    elif pf[i] + l[i] == 2:
        y = 45
    elif pf[i] + pl[i] == 2:
        y = 40
    elif f[i] + mnl[i]== 2:
        y = 35
    elif f[i] + mnr[i]== 2:
        y = 35
    elif l[i] + mn[i]== 2:
        y = 35
    elif f[i] + pmn[i]== 2:
        y = 35
    elif f[i] + pmnl[i]== 2:
        y = 30
    elif pf[i] + mn[i]== 2:
        y = 30
    elif f[i] + d[i]== 2:
        y = 25
    elif l[i] + d[i]== 2:
        y = 25
    elif f[i] + pmnr[i]== 2:
        y = 25
    elif f[i] + pmnl[i]== 2:
        y = 25
    elif pf[i] + pmnl[i]== 2:
        y = 25
    elif pl[i] + mn[i]== 2:
        y = 25
    elif pf[i] + pmnr[i]== 2:
        y = 20
    elif pf[i] + pmnl[i]== 2:
        y = 20
    elif l[i] + pmn[i]== 2:
        y = 20
    elif d[i] + mn[i]== 2:
        y = 15
    elif pf[i] + d[i]== 2:
        y = 15
    elif pl[i] + d[i]== 2:
        y = 15
    elif d[i] + pmn[i]== 2:
        y = 10
    elif d[i] + pmnr[i]== 2:
        y = 5
    elif d[i] + pmnl[i]== 2:
        y = 5
    
    w=x+y+z
    
    weight_sum_2.append(w)

In [39]:
found_2['weight'] = weight_sum_2
found_2

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,0,82,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,2.0,0
1,0,282,0,0,0,0,0,-1000,1,0,0,0,0,0,0,0,0,0.0,-999.0,0
2,0,440,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,3.0,1
3,0,659,0,1,0,0,0,-1000,1,0,0,0,0,0,0,0,0,0.0,-998.0,0
4,0,810,0,1,0,0,0,-1000,1,0,0,0,0,0,0,0,0,0.0,-998.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129098,20000,9308,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1.0,0
1129099,20000,9585,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1.0,0
1129100,20000,9590,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1.0,0
1129101,20000,9634,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1.0,0


In [40]:
#Only looking at matches with weight more than 0
match_2 = found_2[found_2['weight']> 0]
print(len(match_2))

252324


In [41]:
joins_2 = match_2.sort_values(by = ['Dup Index','weight'], ascending = [True,False])
joins_2

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
225961,4004,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1.0,5.0,36
16980,314,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0.0,2.0,1
48732,886,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0.0,2.0,1
80833,1422,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0.0,2.0,1
202455,3605,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801213,14269,10137,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
1128969,19997,10137,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
1129049,19998,10138,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,4.0,35
970225,17161,10138,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25


In [42]:
#Finding unique duplicate records that match with main record
#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
joins_2 = joins_2.reset_index(drop = True)
unq_2, unq_idx_2, unq_cnt_2 = np.unique(joins_2['Dup Index'], return_inverse=True, return_counts=True)
cnt_mask_2 = unq_cnt_2 == 1
unq_ids_2 = unq_2[cnt_mask_2]

#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
cnt_idx_2, = np.nonzero(cnt_mask_2)
idx_mask_2 = np.in1d(unq_idx_2, cnt_idx_2)
idx_idx_2 = np.nonzero(idx_mask_2)
unq_idx_2 = idx_idx_2

In [43]:
#Dropping matched duplicates
to_drop = []
for i in range(len(unq_idx_2)):
    to_drop.append(unq_idx_2[i])

In [44]:
filler = joins_2
for ind in range(len(to_drop[0])):
    index = to_drop[0][ind]
    i = filler['Dup Index'].loc[index]
    main_id[i][0] = filler['Main Index'].loc[index]
    iteration[i] = 3
    unique[i] = 1
    id_count[i] = 1
    quality[i][0] = filler['weight'].loc[index]

In [45]:
c = 0
for i in range(len(id_count)):
    if id_count[i] == 1:
        c = c+ 1

print (c)

8058


In [46]:
joins_2 = joins_2.drop(index = to_drop[0])
joins_2 = joins_2.reset_index(drop = True)

In [47]:
#looking at records where the date of birth match
joins_3 = joins_2[joins_2['DOB']==1]
joins_3

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,4004,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1.0,5.0,36
20,4008,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1.0,5.0,36
23,4017,3,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
45,4021,4,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,1.0,6.0,37
63,4022,5,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249154,19990,10130,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,4.0,35
249156,19991,10131,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,4.0,35
249162,19994,10134,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,4.0,35
249167,19997,10137,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25


In [48]:
#Finding unique duplicate records that match with main record
#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
joins_3 = joins_3.reset_index()
unq_2, unq_idx_2, unq_cnt_2 = np.unique(joins_3['Dup Index'], return_inverse=True, return_counts=True)
cnt_mask_2 = unq_cnt_2 == 1
unq_ids_2 = unq_2[cnt_mask_2]

#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
cnt_idx_2, = np.nonzero(cnt_mask_2)
idx_mask_2 = np.in1d(unq_idx_2, cnt_idx_2)
idx_idx_2 = np.nonzero(idx_mask_2)
unq_idx_2 = idx_idx_2

In [49]:
#Dropping matched duplicates
to_drop = []
for i in range(len(unq_idx_2)):
    to_drop.append(unq_idx_2[i])


In [50]:
c = 0
before = []
for i in range(len(id_count)):
    if id_count[i] == 1:
        c = c+ 1
        before.append(i)
print (c)

8058


In [51]:
filler = joins_3
for ind in range(len(to_drop[0])):
    index = to_drop[0][ind]
    i = filler['Dup Index'].loc[index]
    main_id[i][0] = filler['Main Index'].loc[index]
    iteration[i] = 4
    unique[i] = 1
    id_count[i] = 1
    quality[i][0] = filler['weight'].loc[index]

In [52]:
left = []
for i in range(len(id_count)):
    if id_count[i] == 0:
        left.append(i)

ind_del = []
for i in range(joins_2.shape[0]):
    if joins_2['Dup Index'].loc[i] not in left:
        ind_del.append(i)
        
joins_2 = joins_2.drop(index = ind_del)
joins_2 = joins_2.reset_index(drop = True)
joins_2

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,4401,101,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
1,14184,101,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,3.0,25
2,14654,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,3.0,25
3,3482,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
4,9981,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,8184,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1507,9611,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1508,16041,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1509,17615,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25


In [53]:
#Finding unique duplicate records that match with main record
#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
joins_2 = joins_2.reset_index(drop = True)
unq_2, unq_idx_2, unq_cnt_2 = np.unique(joins_2['Dup Index'], return_inverse=True, return_counts=True)
cnt_mask_2 = unq_cnt_2 == 1
unq_ids_2 = unq_2[cnt_mask_2]

#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
cnt_idx_2, = np.nonzero(cnt_mask_2)
idx_mask_2 = np.in1d(unq_idx_2, cnt_idx_2)
idx_idx_2 = np.nonzero(idx_mask_2)
unq_idx_2 = idx_idx_2

In [54]:
#Dropping matched duplicates
to_drop = []
for i in range(len(unq_idx_2)):
    to_drop.append(unq_idx_2[i])

In [55]:
filler = joins_2
for ind in range(len(to_drop[0])):
    index = to_drop[0][ind]
    i = filler['Dup Index'].loc[index]
    main_id[i][0] = filler['Main Index'].loc[index]
    iteration[i] = 5
    unique[i] = 1
    id_count[i] = 1
    quality[i][0] = filler['weight'].loc[index]

In [56]:
left = []
for i in range(len(id_count)):
    if id_count[i] == 0:
        left.append(i)

ind_del = []
for i in range(joins_2.shape[0]):
    if joins_2['Dup Index'].loc[i] not in left:
        ind_del.append(i)

In [57]:
joins_2 = joins_2.drop(index = ind_del)
joins_2 = joins_2.reset_index(drop = True)
joins_2

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,4401,101,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
1,14184,101,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,3.0,25
2,14654,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,3.0,25
3,3482,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
4,9981,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,8184,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1507,9611,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1508,16041,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1509,17615,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25


In [58]:
#looking at records where the date of birth match
joins_3 = joins_2[joins_2['DOB']==1]
joins_3

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,4401,101,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
2,14654,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,3.0,25
46,4574,152,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1.0,5.0,36
47,15937,152,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,4.0,35
64,4863,212,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,1.0,6.0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,19761,9901,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
1501,16586,9956,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
1502,19816,9956,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
1503,2393,10106,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25


In [59]:
#Finding unique duplicate records that match with main record
#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
joins_3 = joins_3.reset_index(drop = True)
unq_2, unq_idx_2, unq_cnt_2 = np.unique(joins_3['Dup Index'], return_inverse=True, return_counts=True)
cnt_mask_2 = unq_cnt_2 == 1
unq_ids_2 = unq_2[cnt_mask_2]

#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
cnt_idx_2, = np.nonzero(cnt_mask_2)
idx_mask_2 = np.in1d(unq_idx_2, cnt_idx_2)
idx_idx_2 = np.nonzero(idx_mask_2)
unq_idx_2 = idx_idx_2

In [60]:
#Dropping matched duplicates
to_drop = []
for i in range(len(unq_idx_2)):
    to_drop.append(unq_idx_2[i])

In [61]:
filler = joins_3
for ind in range(len(to_drop[0])):
    index = to_drop[0][ind]
    i = filler['Dup Index'].loc[index]
    main_id[i][0] = filler['Main Index'].loc[index]
    iteration[i] = 6
    unique[i] = 1
    id_count[i] = 1
    quality[i][0] = filler['weight'].loc[index]

In [62]:
left = []
for i in range(len(id_count)):
    if id_count[i] == 0:
        left.append(i)

ind_del = []
for i in range(joins_2.shape[0]):
    if joins_2['Dup Index'].loc[i] not in left:
        ind_del.append(i)

In [63]:
joins_2 = joins_2.drop(index = ind_del)
joins_2 = joins_2.reset_index(drop = True)
joins_2

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,4401,101,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
1,14184,101,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,3.0,25
2,14654,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,3.0,25
3,3482,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
4,9981,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,8184,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1507,9611,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1508,16041,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1509,17615,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25


In [64]:
#looking at records with high weight
joins_3 = joins_3[joins_3['weight'] > 45]

In [65]:
#Finding unique duplicate records that match with main record
#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
joins_3 = joins_3.reset_index(drop = True)
unq_2, unq_idx_2, unq_cnt_2 = np.unique(joins_3['Dup Index'], return_inverse=True, return_counts=True)
cnt_mask_2 = unq_cnt_2 == 1
unq_ids_2 = unq_2[cnt_mask_2]

#https://stackoverflow.com/questions/25264798/checking-for-and-indexing-non-unique-duplicate-values-in-a-numpy-array
cnt_idx_2, = np.nonzero(cnt_mask_2)
idx_mask_2 = np.in1d(unq_idx_2, cnt_idx_2)
idx_idx_2 = np.nonzero(idx_mask_2)
unq_idx_2 = idx_idx_2

In [66]:
#Dropping matched duplicates
to_drop = []
for i in range(len(unq_idx_2)):
    to_drop.append(unq_idx_2[i])

In [67]:
#Adding the matched records
filler = joins_3
for ind in range(len(to_drop[0])):
    index = to_drop[0][ind]
    i = filler['Dup Index'].loc[index]
    main_id[i][0] = filler['Main Index'].loc[index]
    iteration[i] = 7
    unique[i] = 1
    id_count[i] = 1
    quality[i][0] = filler['weight'].loc[index]

In [68]:
left = []
for i in range(len(id_count)):
    if id_count[i] == 0:
        left.append(i)

ind_del = []
for i in range(joins_2.shape[0]):
    if joins_2['Dup Index'].loc[i] not in left:
        ind_del.append(i)

In [69]:
joins_2 = joins_2.drop(index = ind_del)
joins_2 = joins_2.reset_index(drop = True)
joins_2

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,4401,101,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
1,14184,101,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,3.0,25
2,14654,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,3.0,25
3,3482,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
4,9981,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,2.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,8184,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1465,9611,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1466,16041,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
1467,17615,10106,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25


In [70]:
joins_2 = joins_2.sort_values(by = ['Dup Index','weight', 'sum'], ascending = [True,False, False])

In [71]:
#Lookint at date of birth exactly matching
joins_3 = joins_2[joins_2['DOB']==1]

In [72]:
joins_3

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,4401,101,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1.0,6.0,37
2,14654,101,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,3.0,25
46,4574,152,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1.0,5.0,36
47,15937,152,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,4.0,35
64,4863,212,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,1.0,6.0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,19761,9901,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
1459,16586,9956,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
1460,19816,9956,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25
1461,2393,10106,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,3.0,25


In [73]:
unq = np.unique(joins_3['Dup Index'])
unq

array([  101,   152,   212,   223,   229,   488,   647,   664,   750,
         757,   831,   854,   863,   873,   907,   918,  1083,  1108,
        1232,  1295,  1392,  1410,  1589,  1708,  1749,  1763,  1792,
        1797,  1815,  1858,  1907,  2044,  3429,  3872,  4329,  4727,
        4805,  6664,  7203,  7228,  7862,  8306,  8773,  8988,  9112,
        9573,  9695,  9755,  9901,  9956, 10106], dtype=int64)

In [74]:
cur = joins_3[joins_3['Dup Index']==unq[0]].reset_index(drop = True)
choose = cur.loc[0]
type(choose['Dup Index'])

numpy.float64

In [75]:
#Adding unique matches
for j in range(len(unq)):
    cur = joins_3[joins_3['Dup Index']==unq[j]].reset_index(drop = True)
    choose = cur.loc[0]
    i = int(choose['Dup Index'])
    main_id[i][0] = int(choose['Main Index'])
    iteration[i] = 8
    unique[i] = 0
    id_count[i] = 1
    quality[i][0] = int(choose['weight'])

In [76]:
#Seeing the records which are left
left = []
for i in range(len(id_count)):
    if id_count[i] == 0:
        left.append(i)

ind_del = []
for i in range(joins_2.shape[0]):
    if joins_2['Dup Index'].loc[i] not in left:
        ind_del.append(i)

In [77]:
joins_2 = joins_2.drop(index = ind_del)
joins_2 = joins_2.reset_index(drop = True)


In [78]:
joins_2.sort_values(by = ['weight'], ascending = False)

,Main Index,Dup Index,Last Name,First Name,Mother's Birth Last Name,Main-Mother->Last,Main-Last->Mother,Phon Last Name,Phon First Name,Phon Mother's Birth Last Name,Phon Main-Mother->Last,Phon Main-Last->Mother,Race,Religion,Citizenship,Language,DOB,fuzzy DOB,sum,weight
0,3185,8152,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
1,6696,8152,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
32,12117,8785,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
33,13425,8785,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
34,19571,8785,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,5147,8731,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0,2.0,25
60,13754,9758,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0.0,3.0,25
24,14960,8696,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,2.0,15
5,11603,8152,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,2.0,15


In [79]:
c = 0
for i in range(len(id_count)):
    if id_count[i] == 1:
        c = c+ 1
print (c)

10061


In [80]:
#test = joins_2[joins_2['Dup Index'] == 8202]
#test

In [81]:
clean_dup.loc[8152]

Phon Last Name                      KR
Phon First Name                    NaN
Phon Mother Name                   NaN
ID                            7b7c94f9
Last Name                          Cro
First Name                         NaN
Date of Birth               01/27/2013
Mother's Birth Last Name           NaN
Indigenous Identity                NaN
Religion                           NaN
Ethnicity                          NaN
Race                               NaN
Place of Birth                     NaN
Citizenship                        NaN
Language (Mother Tongue)           NaN
Name: 8152, dtype: object

In [82]:
clean_main.loc[11646]

Phon Last Name                            KR
Phon First Name                           PL
Phon Mother Name                          KR
ID                                  811aa403
Last Name                               Gray
First Name                              Paul
Date of Birth                     06/12/2011
Mother's Birth Last Name                Gray
Indigenous Identity         Middle Easterner
Religion                               Islam
Ethnicity                   Middle Easterner
Race                        Middle Easterner
Place of Birth                        Angola
Citizenship                           Canada
Language (Mother Tongue)             English
Name: 11646, dtype: object

In [83]:
m_i = []
q = []
for i in range(len(id_count)):
    m_i.append(main_id[i][0])
    q.append(quality[i][0])

In [84]:
#Creating new columns to make a joined main and duplicate record
main_last = []
main_first = []
main_mother = []
main_dob = []
main_race = []
main_lang = []
main_citizen = []
main_reli = []
main_iter = []
main_qual = []
main_index = []

for j in range(len(id_count)):
    if m_i[j] == -1:
        main_last.append('')
        main_first.append('')
        main_mother.append('')
        main_dob.append('')
        main_race.append('')
        main_lang.append('')
        main_citizen.append('')
        main_reli.append('')
        main_index.append('')
        main_iter.append(-1)
        main_qual.append(-1)
        
    else:
        cur = main_org.loc[m_i[j]]
        main_last.append(cur['Last Name'])
        main_first.append(cur['First Name'])
        main_mother.append(cur["Mother's Birth Last Name"])
        main_dob.append(cur['Date of Birth'])
        main_race.append(cur['Race'])
        main_lang.append(cur['Language (Mother Tongue)'])
        main_citizen.append(cur['Citizenship'])
        main_reli.append(cur['Religion'])
        main_index.append(cur['ID'])
        main_iter.append(iteration[j])
        main_qual.append(q[j])

In [85]:
main_id[0]

[4004]

In [86]:
#Combining the records
combined = dup_org.copy()

In [87]:
combined['Main ID'] = main_index
combined['Main Last Name'] = main_last
combined['Main First Name'] = main_first
combined["Main Mother's Birth Last Name"] = main_mother
combined['Main Date of Birth'] = main_dob
combined['Main Language (Mother Tongue)'] = main_lang
combined['Main Citizenship'] = main_citizen
combined['Main Religion'] = main_reli
combined['Quality'] = main_qual
combined['Iteration']  = main_iter   

In [88]:
combined

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,...,Main ID,Main Last Name,Main First Name,Main Mother's Birth Last Name,Main Date of Birth,Main Language (Mother Tongue),Main Citizenship,Main Religion,Quality,Iteration
0,73c2ab6a,NaN,Steven,01/13/2008,NaN,NaN,NaN,Indian,NaN,Canada,...,ed68be4d,Tolbert,Steven,NaN,01/13/2008,NaN,Canada,NaN,36,4
1,488f1063,NaN,Dorothy,05/02/2012,NaN,NaN,Jewish,NaN,NaN,NaN,...,e73ea697,Boling,Dorothy,NaN,05/02/2012,NaN,NaN,Jewish,36,4
2,331bea71,Yerian,Ellwood,08/18/2019,NaN,NaN,NaN,NaN,NaN,NaN,...,24d31a35,Yerian,Elwood,NaN,08/18/2019,English,Canada,Islam,71,3
3,ac3aaf9c,NaN,Robin,08/25/2013,NaN,NaN,NaN,Indian,NaN,Canada,...,50cf0439,Robinson,Robin,NaN,08/25/2013,English,Canada,NaN,37,4
4,ae9c2ae4,NaN,Raymond,05/25/2017,NaN,NaN,Other Christianity,NaN,NaN,Canada,...,a0d1f7c1,Carmona,Raymond,NaN,05/25/2017,English,Canada,Other Christianity,37,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10135,08606dd8,Avila,NaN,06/24/2014,NaN,NaN,NaN,NaN,NaN,NaN,...,70311583,Avila,Steve,NaN,06/24/2014,NaN,Canada,NaN,35,3
10136,8507e3f4,Fernandez,NaN,08/07/2015,NaN,NaN,NaN,NaN,NaN,NaN,...,43d9ceca,Fernandez,Matthew,NaN,08/07/2015,Arabic,Canada,NaN,35,3
10137,d5285223,Mathews,NaN,02/28/2015,NaN,NaN,NaN,NaN,NaN,NaN,...,56c09f74,Matthews,Shari,NaN,02/28/2015,NaN,Canada,Other/not recorded,25,4
10138,7157d300,Konty,NaN,12/15/2022,NaN,NaN,NaN,NaN,NaN,NaN,...,25e05eb1,Konty,Lisa,NaN,12/15/2022,English,NaN,NaN,35,4


In [ ]:
#Exporting the table
combined.to_excel('Combined.xlsx')